In [ ]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, filterFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

# login to myplant()
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

In [ ]:
lookup = 'Forsa Hartmoor'
motor_num = 2

In [ ]:
def sfun(x):
    #return all([ (lookup in str(x['Design Number'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    return all([ (lookup in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')

In [ ]:
ddl = pd.DataFrame(fleet['serialNumber'] + ' - ' + fleet['IB Site Name'] + ' ' + fleet['Engine ID'])[0].to_list()
ddl = [m for m in ddl]
fleet.T;

In [ ]:
motor = fleet.iloc[motor_num]
modes = ['undefined','OFF','MAN','AUTO']
#success = [True,False]
success = [True]
e=Engine.from_fleet(mp,motor)
pp_from=e['Commissioning Date']
#pp_from='2022-03-28 06:00'
#pp_to='2022-03-28 08:14'
pp_to=datetime.now()
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:
# Run State Machines
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm.run2(silent = False)

In [ ]:
rdf = fsm.starts
rda = rdf[:].reset_index(drop='index')
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))].reset_index(drop='index')
rdb = rda
#rda[filterFSM.run2filter_content].round(2)
print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == True].shape[0]}, Failed: {rdf[rdf['success'] == False].shape[0]} => {rdf[rdf['success'] == True].shape[0]/rdf.shape[0]*100.0:3.1f}%")
pd.DataFrame.from_dict(e.dash, orient='index').T

In [ ]:
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success']) & (rda.targetload > 2800.0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda.count_alarms > 0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success'])].copy()
rde = rda
rde['datetime'] = pd.to_datetime(rde['starttime'])
rde['isuccess'] = rde.apply(lambda x: 1 if x['success'] else 0, axis=1)
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout','isuccess']
display(rde[vec].describe().round(2))
dfigsize = (28,12)
dset = [
    {'col':['cumstarttime'],'_ylim':(-600,800), 'color':'darkblue'},
    {'col':['runout'],'_ylim':(0,100) },
    {'col':['targetload'],'_ylim':(-4000,26000) },
    {'col':['ramprate'],'_ylim':(-5,7)},
    {'col':['loadramp'],'_ylim':(-150,900), 'color':'red'},
    {'col':['speedup'],'_ylim':(-100,200), 'color':'orange'},
    {'col':['synchronize'],'_ylim':(-20,400)},
    {'col':['startpreparation'],'_ylim':(-1000,800)},
    {'col':['count_warnings','count_alarms','isuccess'],'_ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,128,0,0.2)'] },
    {'col':['no'],'_ylim':(0,1000), 'color':['rgba(0,0,0,0.1)'] },
    #{'col':['count_warnings','count_alarms','no'],'ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,0,0,0.1)'] }
    ]
dset = equal_adjust(dset, rde, do_not_adjust=[-1], debug=True)
ftitle = f"{fsm._e}"
fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize ,title=ftitle);
bokeh_show(fig)

In [ ]:
out = widgets.Output()
pfigsize=(18,10)

def myfigures(e):
    return {
    'actors' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['TecJet_GasPress1'],'_ylim': [0, 3], 'color':'rgba(255,0,0,0.4)', 'unit':'mbar'},
        {'col':['TecJet_GasTemp1'],'_ylim': [0, 3], 'color':'rgba(255,0,255,0.4)', 'unit':'°C'},
        {'col':['TecJet_GasDiffPress'],'_ylim': [0, 3], 'color':'rgba(0,255,0,0.4)', 'unit':'mbar'},
        {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
        ],
        'tecjet' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['TecJet_GasPress1'],'_ylim': [0, 3], 'color':'rgba(255,0,0,0.4)', 'unit':'mbar'},
        {'col':['TecJet_GasTemp1'],'_ylim': [0, 3], 'color':'rgba(255,0,255,0.4)', 'unit':'°C'},
        {'col':['TecJet_GasDiffPress'],'_ylim': [0, 3], 'color':'rgba(0,255,0,0.4)', 'unit':'mbar'},
        ],
        'lubrication' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
        {'col':['Hyd_PressOilDif'],'ylim': [0, 3], 'color':'black', 'unit': 'bar'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
        {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
        ],
        'exhaust' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
        {'col':e.dataItemsCyl('Knock_Valve_Noise_Cyl*'),'ylim': [0, 12000], 'unit':'mV'},
        ],
        'ignition' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
        {'col':e.dataItemsCyl('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
        {'col':e.dataItemsCyl('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-30, 60], 'unit':'%'},
        ],    
    }

def update_fig(x=0):
    global vv; 
    rdbs = rdb[rdb.no == x]
    if not rdbs.empty:
        startversuch = rdbs.iloc[0]
        #startversuch = rdb.iloc[x]
        vv = startversuch.no; 
        #ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
        #display(HTML(ftitle));
        disp_result(startversuch)

        # PLotter
        lfigures = myfigures(fsm._e)
        plotdef, vset = cplotdef(mp, lfigures)
        #dmaxlength = 1800
        dmaxlength = None
        dminlength = None
        #startversuch = rdb.iloc[vv]
        ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
        data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        fig_handles = []
        for doplot in plotdef:
            dset = lfigures[doplot]
            ltitle = f"{ftitle} | {doplot}"
            if count_columns(dset) > 12: # no legend, if too many lines.
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, legend=False, figsize=pfigsize)
            else:
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, figsize=pfigsize)

            fig = FSM_add_Notations(fig, fsm, startversuch)
            disp_alarms(startversuch)
            disp_warnings(startversuch)
            fig = FSM_add_Alarms(fig, fsm, startversuch)
            fig = FSM_add_Warnings(fig, fsm, startversuch)
            fig_handles.append(bokeh_show(fig, notebook_handle=True))
        for h in fig_handles:
            push_notebook(handle=h)
    else:
        print(f"Start No {x} is not in the filtered results. please consider changing filters, if you want to access it.")


In [ ]:
vv = 0
@out.capture(clear_output=True)
def cb(but):
    global vv; vv = a.value; update_fig(a.value)

a = widgets.IntText(description='No: ',layout=widgets.Layout(width='10%'))
b = widgets.IntSlider(0, 0, rdf.shape[0]-1 , 1, layout=widgets.Layout(width='70%'))
mylink = widgets.jslink((a, 'value'), (b, 'value'))
d = widgets.Button(description='Show Plots',disabled=False, button_style='primary')
d.on_click(cb)
display( widgets.VBox([widgets.HBox([b, a, d]), out]));

In [ ]:
#update_fig(vv)

In [ ]:
startversuch = rdf.iloc[vv]
print("messages leading to state Switches:")
print("-----------------------------------")
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
    print(f"{i:3} {v}")
print(f"\nall messages during start attempt No.:{vv:4d} leading to state Switches:")
print("---------------------------------------------------------------------")
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

In [ ]:
mfn = e._fname + '_messages.txt'
#fsm.save_messages(mfn)
print(mfn)

In [ ]:
vv = 0

'''
z = widgets.Combobox(
    value=ddl[0],
    placeholder='Select Engine',
    options=ddl,
    description='Engine:',
    ensure_option=True,
    disabled=False,
    layout=widgets.Layout(width='60%')
)
'''

a = widgets.IntText(description='No: ',layout=widgets.Layout(width='10%'))
b = widgets.IntSlider(0, 0, rdf.shape[0]-1 , 1, layout=widgets.Layout(width='50%'))
mylink = widgets.jslink((a, 'value'), (b, 'value'))

'''
b1 = widgets.DatePicker(
    value=pd.to_datetime(e['Commissioning Date']),
    description='From: ',
    disabled=False
)

b2 =widgets.DatePicker(
    value = date.today(),
    description='To:',
    disabled=False
)

#c = widgets.SelectMultiple(
#    options=['Apples', 'Oranges', 'Pears'],
#    value=['Oranges'],
#    #rows=10,
#    description='Fruits',
#    disabled=False
#)
'''

d = widgets.Button(
    description='Show Figures',
    disabled=False,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    #icon='check', # (FontAwesome names without the `fa-` prefix)
)


@out.capture(clear_output=True)
def cb(but):
    #print(b.value, but)
    global vv
    vv = a.value
    update_fig(a.value)
d.on_click(cb)

display(
    widgets.VBox(
        [
        #z, 
        #widgets.HBox([b1, b2]),
        widgets.HBox([b, a, d]), 
        out
        ]
    )
);